In [2]:
#!pip install imutils
from imutils import paths 

train_image_files = list(paths.list_images('train'))
test_image_files = list(paths.list_images('test'))
val_image_files = list(paths.list_images('val'))

In [3]:
from tensorflow import keras 
import tensorflow as tf 

# Image preprocessing utils
@tf.function
def parse_images(image_path): 
    """
        this function, will read the image from the directory, and resize it to the 224 image shape, which is predefined 
        image shape of Resnet pretrained model.
        Params:
            image_path(type: str): image filepath in the directory.
        Return(type: tf.Image)
            returns the preprocessed image.
    """
    image_string = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, size=[224, 224])

    return image

def create_tensorflow_dataset(data, batch_size):
    """
        this function, will create the tensorflow dataset from the images.
        Params:
            data(type: List): list of data, in our case list contains the file path.
            batch_size(type: Int): batch size for the generator.
        Return(type: tf.data.Dataset)
            returns tensorflow dataset object.
    """
    tensorflow_data = tf.data.Dataset.from_tensor_slices(train_image_files)
    tensorflow_data = (
    tensorflow_data
        .map(parse_images, num_parallel_calls=tf.data.experimental.AUTOTUNE)
        .shuffle(1024)
        .batch(batch_size, drop_remainder=True)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )
    return tensorflow_data        

In [4]:
train_ds = create_tensorflow_dataset(train_image_files, 32)
test_ds = create_tensorflow_dataset(test_image_files, 32)
val_ds = create_tensorflow_dataset(val_image_files, 32)

In [5]:
import numpy as np
import cv2

class GaussianBlur(object):
    # Implements Gaussian blur as described in the SimCLR paper
    def __init__(self, kernel_size, min=0.1, max=2.0):
        self.min = min
        self.max = max
        # kernel size is set to be 10% of the image height/width
        self.kernel_size = kernel_size

    def __call__(self, sample):
        sample = np.array(sample)

        # blur the image with a 50% chance
        prob = np.random.random_sample()

        if prob < 0.5:
            sigma = (self.max - self.min) * np.random.random_sample() + self.min
            sample = cv2.GaussianBlur(sample, (self.kernel_size, self.kernel_size), sigma)

        return sample
    
gaussiam_blur = GaussianBlur(3) 
# color jitter and grayscale 
def random_color_distortion(image):
    # Random color jittering (strength 0.5)
    color_jitter = tf.random.uniform(shape=[])
    if color_jitter < 0.8:
        image = tf.image.random_brightness(image, max_delta=0.4)
        image = tf.image.random_contrast(image, lower=0.6, upper=1.4)
        image = tf.image.random_saturation(image, lower=0.6, upper=1.4)
        image = tf.image.random_hue(image, max_delta=0.1)
        image = tf.clip_by_value(image, 0, 1)

    # Color dropping
    color_drop = tf.random.uniform(shape=[])
    if color_drop < 0.2:
        image = tf.image.rgb_to_grayscale(image)
      #  image = tf.tile(image, [1, 1, 3])

    return image

def simclr_dataaugumentation_pipeline(images):
    images = tf.image.random_crop(value=images, size=(32, 100, 100, 3))
    images = tf.image.random_flip_left_right(images, seed=None)
    images = tf.image.resize(images, [224, 224])
#    images = random_color_distortion(images)
  #  images = apply_blur(images)
 #       
    return images

In [6]:
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense, Input, Lambda, Input, Conv2D, MaxPooling2D, Activation, GlobalAveragePooling2D

class ProjectionHeadLayer(Layer): 
    def __init__(self, hidden_neurons=128, activation_func=None):
        super(ProjectionHeadLayer, self).__init__()
        self.hidden_neurons = hidden_neurons
        self.activation_func = activation_func
        self.dense_layer = Dense(self.hidden_neurons)
        self.act_func = Activation(self.activation_func)
        
    def call(self, tensor): 
        x = self.dense_layer(tensor)
        x = self.act_func(x)
        
        return x

In [7]:
class ProjectionHead(keras.models.Model): 
    def __init__(self):
        super(ProjectionHead, self).__init__()        
        self.projection_layer_1 = ProjectionHeadLayer(256, activation_func="relu")
        self.projection_layer_2 = ProjectionHeadLayer(128, activation_func="relu")
        self.projection_layer_3 = ProjectionHeadLayer(50, activation_func=None)
        
    def call(self, tensor): 
        x = self.projection_layer_1(tensor)
        x = self.projection_layer_2(x)
        x = self.projection_layer_3(x)
        
        return x    
    
def base_encoder(input_dims, include_top, weights): 
    base_model = tf.keras.applications.ResNet50(include_top=include_top,
                                        weights=weights, input_shape=input_dims)
    base_model.trainable = True
    inputs = Input(input_dims)
    h = base_model(inputs, training=True)
    return keras.models.Model(inputs, h)
        

In [8]:
class BaseEncoder(keras.models.Model):
    def __init__(self, input_dims, include_top, weights): 
        super(BaseEncoder, self).__init__()
        self.input_dims = input_dims
        self._include_top = include_top
        self._weights = weights
        
        self.base_encoder = base_encoder(self.input_dims , self._include_top, self._weights)
        self.projectionhead_model = ProjectionHead()
        self.global_avgpooling = GlobalAveragePooling2D()
        
    def call(self, tensor): 
        x = self.base_encoder(tensor, training=True)
        x = self.global_avgpooling(x)
        x = self.projectionhead_model(x)
        
        return x 

In [9]:
import tensorflow as tf

cosine_sim_1d = tf.keras.losses.CosineSimilarity(axis=1, reduction=tf.keras.losses.Reduction.NONE)
cosine_sim_2d = tf.keras.losses.CosineSimilarity(axis=2, reduction=tf.keras.losses.Reduction.NONE)


def _cosine_simililarity_dim1(x, y):
    v = cosine_sim_1d(x, y)
    return v


def _cosine_simililarity_dim2(x, y):
    # x shape: (N, 1, C)
    # y shape: (1, 2N, C)
    # v shape: (N, 2N)
    v = cosine_sim_2d(tf.expand_dims(x, 1), tf.expand_dims(y, 0))
    return v


def _dot_simililarity_dim1(x, y):
    # x shape: (N, 1, C)
    # y shape: (N, C, 1)
    # v shape: (N, 1, 1)
    v = tf.matmul(tf.expand_dims(x, 1), tf.expand_dims(y, 2))
    return v


def _dot_simililarity_dim2(x, y):
    v = tf.tensordot(tf.expand_dims(x, 1), tf.expand_dims(tf.transpose(y), 0), axes=2)
    # x shape: (N, 1, C)
    # y shape: (1, C, 2N)
    # v shape: (N, 2N)
    return v

def get_delayed_lr(initial_lr, decay_steps): 
    decay_steps = 1000
    lr_decayed_fn = tf.keras.experimental.CosineDecay(initial_learning_rate=initial_lr,
                                                                  decay_steps=decay_steps)
    
    return lr_decayed_fn


def get_negative_mask(batch_size):
    # return a mask that removes the similarity score of equal/similar images.
    # this function ensures that only distinct pair of images get their similarity scores
    # passed as negative examples
    negative_mask = np.ones((batch_size, 2 * batch_size), dtype=bool)
    for i in range(batch_size):
        negative_mask[i, i] = 0
        negative_mask[i, i + batch_size] = 0
    return tf.constant(negative_mask)

In [10]:
def ntxloss(zis, zjs, negatives, batch_size, 
                    criterion, negative_mask, temperature, l_pos): 
    loss = 0
    for positives in [zis, zjs]:
        l_neg = _cosine_simililarity_dim2(positives, negatives)

        labels = tf.zeros(batch_size, dtype=tf.int32)

        l_neg = tf.boolean_mask(l_neg, negative_mask)
        l_neg = tf.reshape(l_neg, (batch_size, -1))
        l_neg /= temperature
        
        logits = tf.concat([l_pos, l_neg], axis=1)         
        loss += criterion(y_pred=logits, y_true=labels)        

    loss = loss / (2 * batch_size)
    return loss

In [11]:
def train_step(ti, tj, model, optimizer, criterion, 
                                batch_size, temperature, negative_mask): 
    with tf.GradientTape() as tape: 
        zis = model(ti)
        zjs = model(tj)
        
        zis = tf.math.l2_normalize(zis, axis=1)
        zjs = tf.math.l2_normalize(zjs, axis=1)

        l_pos = _dot_simililarity_dim1(zis, zjs)
        l_pos = tf.reshape(l_pos, (batch_size, 1))
        l_pos /= temperature

        negatives = tf.concat([zis, zjs], axis=0)
        loss = ntxloss(zis, zjs, negatives, batch_size, 
                               criterion, negative_mask, temperature, l_pos)
    
    params = model.trainable_weights
    grads = tape.gradient(loss, params)
    optimizer.apply_gradients(zip(grads, params))
    
    return loss 

In [13]:
from tqdm import tqdm

def train(model, train_ds, optimizer, criterion, temperature, 
                                  epochs, batch_size, negative_mask): 
    epochs_loss = []
    for epoch in tqdm(range(epochs)):
        epoch_loss = 0
        total_iteration = 0
        for training_data_batch in train_ds: 
            tis = simclr_dataaugumentation_pipeline(training_data_batch)
            tjs = simclr_dataaugumentation_pipeline(training_data_batch)
                        
            loss = train_step(tis, tjs, model, optimizer, criterion, 
                                      batch_size, temperature, negative_mask)
            
            epoch_loss += loss 
            total_iteration += 1 
        
        mean_epoch_loss = epoch_loss / total_iteration
        epochs_loss.append(mean_epoch_loss)
        
        print(f'Epoch; {epoch} loss: {mean_epoch_loss}')
    
    return epochs_loss, model

In [17]:
base_encoder_model = BaseEncoder((224, 224, 3), False, None)
criterion = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, 
                                                          reduction=tf.keras.losses.Reduction.SUM)

optimizer = tf.keras.optimizers.SGD(get_delayed_lr(0.1, 1000))
negative_mask = get_negative_mask(32)

train(base_encoder_model, train_ds, optimizer, criterion, 0.1, 150, 32, negative_mask)

  1%|          | 1/150 [00:59<2:26:55, 59.16s/it]

Epoch; 0 loss: 1.7672856245098956e-07


  1%|▏         | 2/150 [01:52<2:17:27, 55.72s/it]

Epoch; 1 loss: 1.6903503308185464e-07


  2%|▏         | 3/150 [02:45<2:13:19, 54.42s/it]

...

 99%|█████████▉| 149/150 [2:16:58<00:55, 55.07s/it]

Epoch; 148 loss: 1.6931846857914934e-07


100%|██████████| 150/150 [2:17:53<00:00, 55.16s/it]

Epoch; 149 loss: 1.7140382624347694e-07


([<tf.Tensor: shape=(), dtype=float32, numpy=1.7672856e-07>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.6903503e-07>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.7255786e-07>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.7188974e-07>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.7557453e-07>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.6927798e-07>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.7330696e-07>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.6607909e-07>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.6818468e-07>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.7646536e-07>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.7743717e-07>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.684074e-07>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.6717239e-07>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.7306401e-07>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.6903503e-07>,
  <tf.Tensor: shape=(), dtype=float32, numpy=1.7466346e-07>,
  <tf.Tensor: shape=(), d

In [18]:
base_encoder_model.save("simclr_model_1")
keras.backend.clear_session() 